In [1]:
import numpy as np
import random
import torch
from src.dataloaders import Dataset100k
from src.models import GMFBCEModel, MLPBCEModel, NeuralMatrixFactorizationBCEModel
from src.trainer import Trainer
from src.metrics import hitratio, ndcg

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
print(f"{device=}")

device='cuda'


In [2]:
class config:
    data_dir = "ml-100k"
    epochs = 2
    batch_size = 2048
    gmf_embed_size = 16
    mlp_embed_size = 32
    layers = [32, 16, 8]
    lr = 0.001
    b1 = 0.7
    b2 = 0.5
    weight_decay = 0.0001


dataset = Dataset100k(config.data_dir)
dataset.gen_adjacency()
dataset.make_train_test()
print(f"{dataset.train_size=}, {dataset.test_size=}")

metrics = {
    "HR@1": (hitratio, {"top_n": 1}),
    "HR@5": (hitratio, {"top_n": 5}),
    "HR@10": (hitratio, {"top_n": 10}),
    "NDCG@1": (ndcg, {"top_n": 1}),
    "NDCG@5": (ndcg, {"top_n": 5}),
    "NDCG@10": (ndcg, {"top_n": 10}),
}

dataset.train_size=198114, dataset.test_size=943


In [3]:
model = NeuralMatrixFactorizationBCEModel(
    dataset.user_count,
    dataset.item_count,
    gmf_embed_size=config.gmf_embed_size,
    mlp_embed_size=config.mlp_embed_size,
    layers=config.layers,
    alpha=0.5,
)

gmf_model = GMFBCEModel(
    dataset.user_count, dataset.item_count, embed_size=config.gmf_embed_size
)
gmf_model.load_state_dict(torch.load("saved_models/gmfbce.pt"))
mlp_model = MLPBCEModel(
    dataset.user_count,
    dataset.item_count,
    embed_size=config.mlp_embed_size,
    layers=config.layers,
)
mlp_model.load_state_dict(torch.load("saved_models/mlpbce.pt"))

model.load_pretrained_weights(gmf_model, mlp_model)
del gmf_model
del mlp_model

# optimizer = torch.optim.SGD(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.lr,
    betas=(config.b1, config.b2),
    weight_decay=config.weight_decay,
)
# optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

trainer = Trainer(
    dataset,
    model,
    optimizer,
    metrics,
    epochs=config.epochs,
    batch_size=config.batch_size,
    device=device,
)

In [4]:
trainer.train(evaluate=True, verbose=True, progressbar=True)
# trainer.test(verbose=False, pbar=False)

Epoch 0: Avg Loss/Batch 0.309413            


HR@1: 0.29056203605514314
HR@5: 0.6224814422057264
HR@10: 0.7741251325556734
NDCG@1: 0.29056203605514314
NDCG@5: 0.4645940010893288
NDCG@10: 0.5140817324677941


Epoch 1: Avg Loss/Batch 0.307802            


HR@1: 0.2979851537645811
HR@5: 0.6224814422057264
HR@10: 0.7741251325556734
NDCG@1: 0.2979851537645811
NDCG@5: 0.46769246376738405
NDCG@10: 0.5174046509623627


In [5]:
best_epoch = np.argmax([r["NDCG@10"] for r in trainer.test_log])
print(f"{best_epoch}: {trainer.test_log[best_epoch]}")

1: {'HR@1': 0.2979851537645811, 'HR@5': 0.6224814422057264, 'HR@10': 0.7741251325556734, 'NDCG@1': 0.2979851537645811, 'NDCG@5': 0.46769246376738405, 'NDCG@10': 0.5174046509623627}


In [6]:
torch.save(trainer.model.state_dict(), "saved_models/nmfbce.pt")
# trainer.model.load_state_dict(torch.load("saved_models/nmfbce.pt"))